In [1]:
import pandas as pd
import numpy as np
import nltk
import re

# Exploracion de datos

In [2]:
df = pd.read_csv('completeSpamAssassin.csv')
df2= pd.read_csv('enronSpamSubset.csv')

In [3]:
print(df.shape)
df.sample(n=5).head()

(6046, 3)


,Unnamed: 0,Body,Label
5338,5338,I think there's a link to all the details on w...,0
924,924,empty,1
3797,3797,URL: http://scriptingnews.userland.com/backiss...,0
2213,2213,I think that this and other articles confuse S...,0
786,786,The Ultimate Traditional & Internet Marketing ...,1


In [4]:
print(df2.shape)
df2.sample(n=5).head()

(10000, 4)


,Unnamed: 0.1,Unnamed: 0,Body,Label
6659,18776,18776,Subject: a message from jim\n jim has visited ...,0
420,8508,8508,"Subject: logo , stationer , website design and...",1
3089,7188,7188,"Subject: summer macromedia , mlcros 0 ft , sym...",1
6554,25188,25188,Subject: sale to rhone poule meter 988078\n i ...,0
6482,24397,24397,"Subject: re : confidential\n sophie ,\n thanks...",0


In [5]:
df=df[['Body','Label']]
df.sample(n=5).head()

,Body,Label
5741,CVS users: heads up! I think we've been talkin...,0
5562,"FYI,I modified the Mandrake package a few mont...",0
1181,\nMarketingonTarget.com has teamed up \n ...,1
5375,Not that I particuarly enjoy these religious w...,0
4204,URL: http://diveintomark.org/archives/2002/10/...,0


In [6]:
df2=df2[['Body','Label']]
df2.sample(n=5).head()

,Body,Label
5967,Subject: strategic management society conferen...,0
1570,Subject: blow yourr life\n want to know how to...,1
2437,Subject: bluetooth wireless printer adapter @ ...,1
7482,Subject: re : tetco / hpl ( enerfin ) meter # ...,0
9455,Subject: increase mdq for contract 5910\n shar...,0


In [7]:
def remove(d):
    d= d.replace("Subject: ","")
    return d
df2['Body']= df2['Body'].apply(remove)
df2

,Body,Label
0,stock promo mover : cwtd\n * * * urgent invest...,1
1,are you listed in major search engines ?\n sub...,1
2,"important information thu , 30 jun 2005 .\n su...",1
3,= ? utf - 8 ? q ? bask your life with ? =\n = ...,1
4,""" bidstogo "" is places to go , things to do\n ...",1
...,...,...
9995,"monday 22 nd oct\n louise ,\n do you have half...",0
9996,missing bloomberg deals\n stephanie -\n i beli...,0
9997,eops salary survey questionnaire\n we will nee...,0
9998,"q 3 comparison\n hi louise ,\n i have a compar...",0


In [8]:
array = [1, 0]
df2.loc[df2['Label'].isin(array)]

,Body,Label
0,stock promo mover : cwtd\n * * * urgent invest...,1
1,are you listed in major search engines ?\n sub...,1
2,"important information thu , 30 jun 2005 .\n su...",1
3,= ? utf - 8 ? q ? bask your life with ? =\n = ...,1
4,""" bidstogo "" is places to go , things to do\n ...",1
...,...,...
9995,"monday 22 nd oct\n louise ,\n do you have half...",0
9996,missing bloomberg deals\n stephanie -\n i beli...,0
9997,eops salary survey questionnaire\n we will nee...,0
9998,"q 3 comparison\n hi louise ,\n i have a compar...",0


In [9]:
df_merged=pd.concat([df, df2], axis=0)
df_merged


,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1
...,...,...
9995,"monday 22 nd oct\n louise ,\n do you have half...",0
9996,missing bloomberg deals\n stephanie -\n i beli...,0
9997,eops salary survey questionnaire\n we will nee...,0
9998,"q 3 comparison\n hi louise ,\n i have a compar...",0


# Preprocesamiento y normalizacion de texto

In [10]:
def to_string(d):
    d= str(d)
    return d
df_merged['Body']=df_merged['Body'].apply(to_string)
df_merged

,Body,Label
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,##############################################...,1
4,I thought you might like these:\n1) Slim Down ...,1
...,...,...
9995,"monday 22 nd oct\n louise ,\n do you have half...",0
9996,missing bloomberg deals\n stephanie -\n i beli...,0
9997,eops salary survey questionnaire\n we will nee...,0
9998,"q 3 comparison\n hi louise ,\n i have a compar...",0


In [11]:
df_merged.dtypes

Body     object
Label     int64
dtype: object

In [13]:
import unicodedata

stop_words = nltk.corpus.stopwords.words('english')
def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =True):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto)
    return texto
def normalize(doc):
    doc = unicodedata.normalize('NFKD', doc).encode('ascii','ignore').decode('utf-8','ignore')
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc
def norm_corpus(d):
    normalize_corpus = np.vectorize(normalize)

    norm_corpus = normalize_corpus(d)
    return norm_corpus

In [14]:
df_merged['Body']=df_merged['Body'].apply(removerCaracteresEspecialesNumerosSimbolos)
df_merged['Body']=df_merged['Body'].apply(norm_corpus)

df_merged

,Body,Label
0,save life insurance spend tolife quote savings...,1
1,fight risk cancer httpwwwadclickwspcfmospk sli...,1
2,fight risk cancer httpwwwadclickwspcfmospk sli...,1
3,adult club offers free membership instant acce...,1
4,thought might like slim guaranteed lose lbs da...,1
...,...,...
9995,monday nd oct louise half hour free diary see ...,0
9996,missing bloomberg deals stephanie believe foun...,0
9997,eops salary survey questionnaire need establis...,0
9998,q comparison hi louise comparison first two we...,0


# Modelo TFIDF

In [14]:

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(df_merged['Body'])
tv_matrix = tv_matrix.toarray()

vocabulario = tv.get_feature_names_out()
tfidf=pd.DataFrame(tv_matrix, columns=vocabulario)
tfidf

,aa,aaa,aaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxl,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacmmvzcjzld,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuwlcuwmdlo,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabcfudhgkxt,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacuxrmplffhxlmh,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaafcvwyfklcuxrmplf,...,zzzryanairspamassassintaintorg,zzzz,zzzzasonorgmy,zzzzcchackwatchcom,zzzzjmasonorg,zzzzneilgarnercomas,zzzzspamassassintaintorg,zzzzspamassassintaintorgoldhtlheuhcclco,zzzzteana,zzzzteanayahoogroupscom
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.069793,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
16042,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
16043,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
16044,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [15]:
from sklearn.model_selection import train_test_split
x= tfidf
y= df_merged['Label']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.3)


In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [18]:
model = MultinomialNB().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2723
           1       0.97      0.91      0.94      2091

    accuracy                           0.95      4814
   macro avg       0.95      0.94      0.95      4814
weighted avg       0.95      0.95      0.95      4814



In [19]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

Confusion Matrix
 [[2656   67]
 [ 181 1910]]


# Modelo BOG

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=3)

bv_matrix = vectorizer.fit_transform(df_merged['Body'])

bv_matrix = bv_matrix.toarray()
vocabulario = vectorizer.get_feature_names_out()
bog=pd.DataFrame(bv_matrix, columns=vocabulario)
bog

,aa,aa ca,aa cc,aa check,aa corp,aa day,aa end,aa meetings,aa please,aa program,...,zyban zyban,zyban zyrtec,zyrtec,zyrtec hello,zzo,zzzz,zzzzspamassassintaintorg,zzzzspamassassintaintorg password,zzzzteana,zzzzteanayahoogroupscom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split
x= bog
y= df_merged['Label']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.3)


In [20]:
model = MultinomialNB().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2766
           1       0.96      0.96      0.96      2048

    accuracy                           0.97      4814
   macro avg       0.97      0.97      0.97      4814
weighted avg       0.97      0.97      0.97      4814



In [21]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

Confusion Matrix
 [[2683   83]
 [  74 1974]]


# Conclusion
Se puede observar que en este caso el modelo de BoG tuvo un mejor desempeño tanto en la prediccion acertada, como se observa en la matriz de confusion, que sus falsos negativos y positivos, 74 y 83 respepctivamente son menores que para TFIDF que tiene 181 y 67. Tambien en el reporte de clasificacion la presicion aumento para BoG y el f1-score con 0.97 y 0.96 contra el f1-score de TFIDF con 0.96 y 0.94. Una hipotesis que se tiene es que esto se debe a la cantidad de vocabulario generado por BOG que fue alrededor de 160,000 contra el 130,000 de TIFDF, se puede extender este vocabulario variando los paramentros de cada modelo, pero en este caso estos parametros utilizados son debido a que el limite de memoria es bajo, pero con mayor potencia se pueden utilizar mejores y mas acertados parametros. No se considera que un modelo es mejor que otro, sino que son utilizados para diferentes situaciones y experimentos.